In [1]:
%matplotlib inline

In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [3]:
results = pd.read_csv('../data/parsed_data.csv')

In [4]:
results['ice_out'] = pd.to_datetime(results['ice_out'])
results.head()
results1 = results[(results.Year>=1950)&(results.Year<=2016)][['Year', 'JDOY']].copy()
results1.columns = ['Year', 'iceout']

In [5]:
df = pd.read_csv('../data/PANN2.csv', parse_dates=['date'], index_col='date')
df = df.dropna(how='all', axis=1)
df['doy'] = df.index.dayofyear


df['year'] = df.index.year
# push OCT, NOV, and DEC into the appropriate Ice Classic Results Year
df['year'] = np.where(df.index.month == 10, df.year+1, df.year)
df['year'] = np.where(df.index.month == 11, df.year+1, df.year)
df['year'] = np.where(df.index.month == 12, df.year+1, df.year)

df = df.resample('W-MON').median()
df['week'] = df.index.week

In [6]:
df.columns = ['actual_mean_temp', 'wind', 'doy', 'year', 'week']

df_temp = df[['actual_mean_temp', 'doy', 'week', 'year']]
df_wind = df[['wind', 'doy', 'week', 'year']]

In [7]:
df2 = pd.pivot_table(df_temp, values='actual_mean_temp', columns='week', index='year')
df2 = df2.dropna(how='all', axis=1).copy()

df3 = pd.pivot_table(df_wind, values='wind', columns='week', index='year')
df3 = df3.dropna(how='all', axis=1).copy()

In [8]:
col_names = df2.columns.tolist()
col_names_corr = []
for col in col_names:
    col = 'temp'+str(col)
    col_names_corr.append(col)
    
col_names_w = df3.columns.tolist()
col_names_corr_w = []
for col in col_names_w:
    col = 'wind'+str(col)
    col_names_corr_w.append(col)

In [9]:
df2.columns = col_names_corr
df3.columns = col_names_corr_w

In [10]:
df4 = df2.merge(results1, how='inner', left_index=True, right_on='Year')
df5 = df4.merge(df3, how='inner', left_on="Year", right_index=True)
df_hold = df5[(df5.Year<=2016)&(df5.Year>=2013)].copy()
df5 = df5[df5.Year<2014]
df5 = df5.drop(['Year'], axis=1)
df_hold = df_hold.drop(['Year'], axis=1)

In [11]:
df5 = df5.fillna(method='ffill').fillna(method='bfill')

In [12]:
features = df5.columns.tolist()
features = [x for x in features if x != 'iceout']

In [24]:
std_scale = preprocessing.StandardScaler().fit(df5)
df_std = pd.DataFrame(std_scale.transform(df5))
df_std.columns = df5.columns

minmax_scale = preprocessing.MinMaxScaler().fit(df5)
df_minmax = pd.DataFrame(minmax_scale.transform(df5))
df_minmax.columns = df5.columns

In [25]:
y = df_minmax.iceout.values
X = df_minmax[features].values

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [ ]:
tpot = TPOTRegressor(generations=40, population_size=300, verbosity=2, scoring='r2', warm_start=False, cv=9)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

In [ ]:
tpot2 = TPOTRegressor(generations=30, population_size=275, verbosity=2, scoring='mean_squared_error', warm_start=False, cv=3)
tpot2.fit(X_train, y_train)
print(tpot2.score(X_test, y_test))

Optimization Progress:   6%|▋         | 550/8525 [04:14<33:34,  3.96pipeline/s]  

Generation 1 - Current best internal CV score: 0.03778541117976199


Optimization Progress:  10%|▉         | 825/8525 [06:06<24:57,  5.14pipeline/s]  

Generation 2 - Current best internal CV score: 0.03778541117976199


Optimization Progress:  13%|█▎        | 1100/8525 [08:24<26:31,  4.67pipeline/s] 

...

Generation 23 - Current best internal CV score: 0.03053852728687564


Optimization Progress:  81%|████████  | 6875/8525 [1:03:19<08:07,  3.38pipeline/s]   

Generation 24 - Current best internal CV score: 0.03053852728687564


Optimization Progress:  90%|█████████ | 7700/8525 [1:12:50<05:20,  2.58pipeline/s]   

Generation 27 - Current best internal CV score: 0.03053852728687564


In [ ]:
tpot2.export('pipeline.py')

In [ ]:
df_hold['iceout']

In [33]:
tpot2.predict(df_hold[features].iloc[-4].fillna(0))

/home/cwcobb/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 129.12308852])

In [24]:
df_hold['iceout'].iloc[-4]

140.61180555555555